# Preprocessing

In [ ]:
# Import all relevant libraries
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load the csv file and make it a dataframe
df = pd.read_csv("Boliga_Webscraped.csv", delimiter = "\t")

In [ ]:
# Drop irrelevant columns and make a copy out of it
df_copy = df.drop(["Unnamed: 0", "estateId", "change", "guid", "estateCode", "groupKey", "canGetVR", "bfEnr"], axis=1)

In [ ]:
# We see we need to change the soldDate column
# Change data type to string
df_copy["soldDate"]=df_copy["soldDate"].astype("string")
# Remove the last 14 digits
df_copy["soldDate"] = df_copy["soldDate"].str[:-14]

In [ ]:
# We need to make a unique identifier
id = [i+1 for i in range(len(df_copy))]
df_copy["SellID"] = id

In [ ]:
df_copy["ppsm"].dtypes

In [ ]:
# Filling in null values of sqmPrice by creating our own column and removing the old column and round up to 2 decimals
df_copy["ppsm"] = df_copy["price"] / df_copy["size"]
df_copy["ppsm"] = df_copy['ppsm'].round(decimals = 2)
df_copy = df_copy.drop("sqmPrice", axis =1)

In [ ]:
# Filling in the null values of the city
df_copy["city"].fillna("Tunø", inplace = True)
df_copy.head()

In [ ]:
# Check if all null values have been removed
df_copy.isna().sum()

In [ ]:
# Put columns in a nice order
df_copy = df_copy.iloc[:, [13,0,1,12,11,9,10,2,14,3,4,5,6,7,8]]

In [ ]:
# Removing outliers
# Outliers regarding size
for row in tqdm(range(len(df_copy))):
    if df_copy.iloc[row]["size"] < 20 or df_copy.iloc[row]["size"] > 2000 or \
    df_copy.iloc[row]["ppsm"] > 5000 or df_copy.iloc[row]["size"] < 100000 or \
    df_copy.iloc[row]["rooms"] > 0 or df_copy.iloc[row]["rooms"] < 12 or \
    df_copy.iloc[row]["buildYear"] > 2022 or df_copy.iloc[row]["buildYear"] < 0:
        df_copy.drop(df_copy.index[row], inplace=True)


In [ ]:
# Filling in the names of the property type, not the numbers
df_copy["propertyType"]=df_copy["propertyType"].astype("string")
df_copy['propertyType'] = df_copy['propertyType'].replace(['1', '2', '3', '4', '6'], ['Villa', 'Terraced House', 'Condominium', 'Holiday Home','Country Property'])


In [ ]:
# Exporting the dataset to a csv file to get it into SQL
df_copy.to_csv("final_dataset.csv")